In [1]:
from sklearn.metrics import f1_score

In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *
from src.fewshot_ner_viz_component.ne_sim_visualizer import *

2018-08-28 18:10:13.200 INFO in 'summarizer.preprocessing.cleaner'['textcleaner'] at line 37: 'pattern' package not found; tag filters are not available for English
2018-08-28 18:10:13.230 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


In [3]:
NE_TYPES = ['GPE', 'LOC', 'ORG', 'FAC', 'EVENT']
N_EXAMPLES = 5
N_TEST_SENTENCES = 5

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, use_cap_feat=True, use_glove=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-28 18:10:14.189 INFO in 'tensorflow'['tf_logging'] at line 159: Using /tmp/tfhub_modules to cache modules.
2018-08-28 18:10:14.912 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 96: [loading embeddings from `/home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-28 18:10:14.913 INFO in 'gensim.models.utils_any2vec'['utils_any2vec'] at line 170: loading projection weights from /home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-28 18:10:14.913 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-28 18:10:36.498 INFO in 'gensim.models.utils_any2vec'['utils_any2vec'] at line 232: loaded (400000, 100) matrix from /home/clement/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-28 18:10:36.689 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


In [8]:
markup_color = NeSimVisualizer.get_color(244,50,244)
pred_color = [NeSimVisualizer.get_color(255, 128, 0), NeSimVisualizer.get_color(0, 255, 0)]
viz_support = NeSimVisualizer(color=markup_color)
viz_pred = NeSimVisualizer(color=pred_color)

In [9]:
for ne_type in NE_TYPES:
    print('Named entity type: {}'.format(ne_type))
    dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)
    np.random.seed(33)
    tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
    np.random.seed(33)
    tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
    y_test = tags2binaryFlat(tags_test)
    model = FewshotNerBinaryClassifier(embedder)
    model.train_on_batch(tokens_train, tags_train)
    results = model(tokens_test)
    viz_support.display_ne_sim(tokens_train, tags2binaryFlat(tags_train), transform=False, title='')
    viz_pred.display_ne_sim(tokens_test, results['probas'], transform=False, title='')
    f1 = f1_score(tags2binaryFlat(tags_test), results['pred'])
    print('F1 score: {:.2f}'.format(f1*100))

Named entity type: GPE
Num of train sentences: 10644
Num of valid sentences: 1592
Num of test sentences: 1573
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-28 18:12:02.957 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


# ne: 5, # tokens: 173
n_samples: 173
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-28 18:12:04.157 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


SVM classifier model


...

2018-08-28 18:12:21.703 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


# ne: 17, # tokens: 220
n_samples: 220
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-08-28 18:12:23.185 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


SVM classifier model


F1 score: 72.73
